In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu 8-Oct-2021

Link :
    Rnd Forestregression model : https://levelup.gitconnected.com/random-forest-regression-209c0f354c84
    Onehot encoder : https://towardsdatascience.com/columntransformer-in-scikit-for-labelencoding-and-onehotencoding-in-machine-learning-c6255952731b
                     https://stackoverflow.com/questions/58087238/valueerror-setting-an-array-element-with-a-sequence-when-using-onehotencoder-on

@author: 41162395
"""


# from numpy import array
# from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
import seaborn as sns
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
#from sklearn.ensemble import IsolationForest
#from sklearn.preprocessing import MinMaxScaler


In [27]:
# Import raw dataset to pandas dataframe
df_org = pd.read_csv('ballshear.csv')
df = pd.read_csv('ballshear.csv')


In [28]:
# Data cleasing drop NAN columns
to_drop = ['LSL','USL','Parameter.Recipe','PROJECT_TYPE']
df_org.drop(to_drop, inplace=True, axis=1)
df_org.dropna(inplace=True)
df.drop(to_drop, inplace=True, axis=1)
df.dropna(inplace=True)
df_org.dropna(inplace=False)



,C_RISTIC,CUSTOMER,PT,DATE_TIME,SHIFT,EN_NO,DEVICE,REMARK,SUBGRP,WIRE_SIZE,...,Parameter.No,Parameter.BondType,Parameter.No_1,Parameter.Value,Parameter.Max,Parameter.Min,Parameter.CreateTime,Parameter.EquipOpn,Parameter.EquipID,Parameter.ULotID
0,STM0369P_BS_06_RSB,STM,STDPO3895.2,07/23/2021 22:43,Z,409674,N7BB*MV8PBAD,S,5,0.6 MIL GMG TYPE,...,3,BSOB Wire,Release,0,0,0,07/23/2021 7:51,WB,TWBR046,STDPO3809.1
1,STM0369P_BS_06_RSB,STM,STDPO3895.2,07/23/2021 22:43,Z,409674,N7BB*MV8PBAD,S,5,0.6 MIL GMG TYPE,...,1,Normal Wire,Release,0,0,0,07/23/2021 11:51,WB,TWBR046,STDPO3809.1
2,STM0369P_BS_06_RSB,STM,STDPO3895.2,07/23/2021 22:43,Z,409674,N7BB*MV8PBAD,S,5,0.6 MIL GMG TYPE,...,2,BSOB Wire,Release,0,0,0,07/23/2021 15:51,WB,TWBR046,STDPO3809.1
3,STM0369P_BS_06_RSB,STM,STDPO3895.2,07/23/2021 22:43,Z,409674,N7BB*MV8PBAD,S,5,0.6 MIL GMG TYPE,...,3,BSOB Ball,Release,0,0,0,07/23/2021 19:51,WB,TWBR046,STDPO3809.1
4,STM0369P_BS_06_RSB,STM,STDPO3895.2,07/23/2021 22:43,Z,409674,N7BB*MV8PBAD,S,5,0.6 MIL GMG TYPE,...,2,BSOB Wire,Release,0,0,0,07/23/2021 15:53,WB,TWBR046,STDPO3809.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40303,PA30133P_BS_10_NB_D1,PA3,PAPPO6312.1,07/23/2021 17:55,X,367318,AN44143A-VBA_SFG,S,10,1.0mil AuPdCu CLR-1A,...,1,Normal Wire,Contact,0,0,0,07/23/2021 6:59,WB,TWBR126,STDPO1669.2
40304,PA30133P_BS_10_NB_D1,PA3,PAPPO6312.1,07/23/2021 17:55,X,367318,AN44143A-VBA_SFG,S,10,1.0mil AuPdCu CLR-1A,...,1,Normal Wire,Pre Base,0,0,0,07/23/2021 7:59,WB,TWBR126,STDPO1669.2
40305,PA30133P_BS_10_NB_D1,PA3,PAPPO6312.1,07/23/2021 17:55,X,367318,AN44143A-VBA_SFG,S,10,1.0mil AuPdCu CLR-1A,...,1,Normal Wire,Base,0,0,0,07/23/2021 8:59,WB,TWBR126,STDPO1669.2
40306,PA30133P_BS_10_NB_D1,PA3,PAPPO6312.1,07/23/2021 17:55,X,367318,AN44143A-VBA_SFG,S,10,1.0mil AuPdCu CLR-1A,...,1,Normal Wire,Pre Release,0,0,0,07/23/2021 9:59,WB,TWBR126,STDPO1669.2


In [29]:
# Drop unwant columns
to_drop = ['C_RISTIC','DATE_TIME','SHIFT','PT','EN_NO','DEVICE','REMARK','SD',
           'BOM_NO','SUBGRP','PLANT_ID','MC_ID','MC_NO','COUNTER',
            'CHAR_MINOR','PACKAGE','DATE_','CIMprofile.cim_machine_name',
            'Parameter.DataType','Parameter.Unit',
            'Parameter.Valid','Parameter.EquipOpn','Parameter.EquipID',
            'Parameter.ULotID','Parameter.CreateTime']
df.drop(to_drop, inplace=True, axis=1)



In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40308 entries, 0 to 40307
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CUSTOMER            40308 non-null  object 
 1   WIRE_SIZE           40308 non-null  object 
 2   MEANX               40308 non-null  float64
 3   Parameter.Group     40308 non-null  int64  
 4   Parameter.No        40308 non-null  int64  
 5   Parameter.BondType  40308 non-null  object 
 6   Parameter.No_1      40308 non-null  object 
 7   Parameter.Value     40308 non-null  int64  
 8   Parameter.Max       40308 non-null  int64  
 9   Parameter.Min       40308 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 3.1+ MB


In [31]:
# Rearrange column position
cols_to_move = ['Parameter.Max','Parameter.Min','Parameter.Value','MEANX']
new_cols = np.hstack((df.columns.difference(cols_to_move), cols_to_move))
df = df.reindex(columns=new_cols)



In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40308 entries, 0 to 40307
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CUSTOMER            40308 non-null  object 
 1   Parameter.BondType  40308 non-null  object 
 2   Parameter.Group     40308 non-null  int64  
 3   Parameter.No        40308 non-null  int64  
 4   Parameter.No_1      40308 non-null  object 
 5   WIRE_SIZE           40308 non-null  object 
 6   Parameter.Max       40308 non-null  int64  
 7   Parameter.Min       40308 non-null  int64  
 8   Parameter.Value     40308 non-null  int64  
 9   MEANX               40308 non-null  float64
dtypes: float64(1), int64(5), object(4)
memory usage: 3.1+ MB


In [8]:
# Input json for prediction 
#dfx = pd.read_json('x_input.json')  



In [33]:
# One hot encoder to numpy array X as below 9 columns or features
# Parameter.BondType,Parameter.Group,Parameter.No,Parameter.No_1,WIRE_SIZE,Parameter.Max,Parameter.Min,Parameter.Value, MEANX
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(handle_unknown='ignore'), [0,1,2,3,4,5])], remainder='passthrough')
X = ct.fit_transform(df).toarray()  # upper case 'X'
#X0 = ct.transform(dfx).toarray()  # upper case 'X0'


In [35]:
X.shape

(40308, 48)

In [10]:
# Split X to 2 arrays as x inputFeatures and y outputResponse
selector = [i for i in range(X.shape[1]) if i != 47] #Column 35 is 'MEANX' 
x = X[:,selector]  # lower case 'x'  all column except 'MEANX'
y = X[:,47]  # lower case 'y'  for 'MEANX'


In [11]:
# Splitting the dataset into the Training set and Test set"""
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)



In [12]:
# Random Forest Regressor Algorithm 
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=10, random_state=(0))
regressor.fit(X_train, y_train)



RandomForestRegressor(n_estimators=10, random_state=0)

In [13]:
# Predicting the results of the Test set
y_pred = regressor.predict(X_test)
#y_pred_2 = np.ravel(y_pred)  # Reduce shape of y_pred
compare = pd.DataFrame(y_test)
compare['y_pred'] = y_pred.tolist()
compare.to_csv('compare.csv',index=False)



In [14]:
# printing the results of the current iteration
np.set_printoptions(precision=2)
compare = np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)
print(compare)
MAPE = np.mean(100 * (np.abs(y_test-regressor.predict(X_test))/y_test))
print('Accuracy:', 100-MAPE)



[[21.63 22.25]
 [21.53 21.92]
 [21.54 19.56]
 ...
 [31.1  23.67]
 [23.09 21.67]
 [21.56 21.92]]
Accuracy: 90.12762987835039


In [15]:
#add 'Y' array as new column in df DataFrame
PRED = regressor.predict(x)
PRED_2 = np.ravel(PRED)  # Reduce shape of y_pred
df['PRED'] = PRED_2.tolist()
#Export output to csv
df.to_csv('ballshear_regression.csv')


In [ ]:
#############################
# Common command 
#-------------------------
# df.describe
# df.info()
# df.index
# df.columns
# dfx[:5]
# du1=ds1.unstack()
# df.isna().sum()
# dfx = df.set_index(['C_RISTIC','PT','Parameter.CreateTime','Parameter.BondType','Parameter.No'])
# df.loc[25024]
# for i in range (len(Y)):
#     if Y[i] == -1:
#         print (df.loc[i])
#
# selected_columns = df[["col1","col2"]]
# new_df = selected_columns.copy()
#
# sns.scatterplot(x='sepal_length', y='sepal_width', hue='class', data=iris)
# sns.scatterplot(x='MEANX', y='SD', data=temp)
#
# df0 = pd.read_json(jsonstr,orient="index")
# df0 = pd.read_json('input.json',orient="index")
# df0.drop(to_drop, inplace=True)
# df0 = df0.transpose()
# 
# df_org[Y==-1]  #List Row for Y = -1 
#
# df0 = df[:1]  #For dummy prediction of testing data
# df0json = df0.to_json()
# file1 = open("x_input.json","w")
# file1.writelines(df0json)
# file1.close() 
#
# df.groupby('C_RISTIC').count()    #pandas count categories
# df.groupby('Parameter.Group').count()
#
# cpare = pd.DataFrame(compare)    #np array to dataframe
# cpare.to_csv('compare.csv')
#
# df.rename(columns={"A": "a", "B": "c"})

